Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [1]:
import os
import time

import pandas as pd
import numpy as np
from collections import defaultdict

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error as mse

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb

PATH = '/datasets/'
SEED = 21

In [2]:
data = pd.read_csv(os.path.join(PATH, 'autos.csv'),
                   parse_dates=['DateCrawled', 'LastSeen', 'DateCreated'])
data.info()
data.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null datetime64[ns]
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null datetime64[ns]
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(7), object(6)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
331235,2016-03-25 18:57:26,10990,wagon,2010,manual,147,6_reihe,100000,4,petrol,mazda,no,2016-03-25,0,19348,2016-04-07 01:45:16
345917,2016-03-12 20:55:39,14750,sedan,2014,manual,86,golf,20000,11,petrol,volkswagen,no,2016-03-12,0,38440,2016-03-15 08:46:32
132755,2016-03-31 14:50:54,1550,small,2002,manual,60,ka,125000,11,petrol,ford,no,2016-03-31,0,51580,2016-04-06 07:45:17


In [3]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Удалим колонки которые явно не имеют смысла для предсказания цены:
1. `DateCrawled` - дата получения объявления из базы
2. `DateCreated` - дата создания объявления
3. `LastSeen` - последняя активность пользователя
4. `NumberOfPictures` -   количество фото, оно везде равно 0

Дополнительно выделим категориальные колонки

In [4]:
col_to_drop = ['DateCrawled', 'LastSeen', 'DateCreated', 'NumberOfPictures']
data = data.drop(columns=col_to_drop)

cat_col = ['VehicleType', 'Gearbox', 'Model', 'FuelType',
           'Brand', 'RegistrationMonth', 'NotRepaired']

Посмотрим пропуски в оставшихся колонках

In [5]:
def show_na(df):
    data_info = (df.isna() | df.isnull()).sum()
    res = (pd.concat([data_info / df.shape[0], data_info], axis=1,  keys=['percent', 'abs'])
           .sort_values('percent', ascending=False))
    return res

show_na(data)

,percent,abs
NotRepaired,0.200791,71154
VehicleType,0.105794,37490
FuelType,0.092827,32895
Gearbox,0.055967,19833
Model,0.055606,19705
Price,0.000000,0
RegistrationYear,0.000000,0
Power,0.000000,0
Kilometer,0.000000,0
RegistrationMonth,0.000000,0


Заполним пропуски в категориальных фичах текстом 'Nan'

In [6]:
data[cat_col] = data[cat_col].fillna('Nan')

Выделим целевое значение и признаки

Разобьем данные на train и test, в соотношении 4 к 1

Train дополнительно разобьем на train_valid и valid, так же в соотношении 4 к 1

In [7]:
X = data.drop(columns='Price')
y = data['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
X_train_valid, X_valid, y_train_valid, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)

print('Valid shapes:')
print(X_train_valid.shape, X_valid.shape, y_train_valid.shape, y_valid.shape)
print('Train/test shapes:')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Valid shapes:
(226796, 11) (56699, 11) (226796,) (56699,)
Train/test shapes:
(283495, 11) (70874, 11) (283495,) (70874,)


# 2. Обучение моделей

Действовать будем по следующему плану:
1. Напишем функцию которой на вход будут подаваться модель и датасеты и она будет возвращать время обучения, время предсказания и rmse
1. Проверим 3 модели: Ridge, CatBoost, LightGBM.
1. Для каждой моделии найдем лучшие гиперпараметры, и измерим время обучения на train
1. Сведем данные по всем моделям в 1 таблицу



In [8]:
def model_eval(model, X_train, y_train, X_test, y_test):
    
    start = time.time()    
    model.fit(X_train, y_train)
    training_time = time.time() - start
    
    start = time.time()  
    y_pred = model.predict(X_test)
    predict_time = time.time() - start

    return training_time, predict_time, mse(y_test, y_pred)**0.5

#### Ridge
1. Подготовим пайплайн: OHE -> scaler -> ridge
1. На кросс валидации найдем лучшие параметры модели
1. Измерим время обучения и запишем результаты на тесте.

In [25]:
res = []

In [10]:
pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', Ridge(random_state=SEED))
])

params = [
    {
        'model__alpha': np.logspace(-2, 2, 20)
    } 
]


<font color="blue">Ridge - оригинально, не встречал еще эту модель. Лайк. Отдельный лайк что делаешь OHE

In [11]:
cv = KFold(n_splits=3, shuffle=True, random_state=SEED)

grid = GridSearchCV(pipe,
                    param_grid=params,
                    cv=cv,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    verbose=False)

In [12]:
%%time
grid.fit(X_train_valid, y_train_valid);

CPU times: user 4min 16s, sys: 2.9 s, total: 4min 19s
Wall time: 4min 21s


GridSearchCV(cv=KFold(n_splits=3, random_state=21, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('ohe',
                                        OneHotEncoder(categorical_features=None,
                                                      categories=None,
                                                      drop=None,
                                                      dtype=<class 'numpy.float64'>,
                                                      handle_unknown='ignore',
                                                      n_values=None,
                                                      sparse=True)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=False,
                                                       with_std=Tru...
       6.95192796e

In [13]:
grid.best_params_

{'model__alpha': 100.0}

In [26]:
res.append(model_eval(grid.best_estimator_, X_train_valid, y_train_valid, X_valid, y_valid))

#### LightGBM
1. Готовим категориальные признаки через Label Encoder
1. Ищем лучшие параметры по сетке используя Scikit-learn API
1. Обучаем модель на всей train выборке и замеряем время и RMSE

In [33]:
encoder = LabelEncoder()

lgb_train = X_train_valid.copy()
lgb_test = X_valid.copy()
lgb_full_train = X_train.copy()
lgb_full_test = X_test.copy()


cat_col_num = []

for col in cat_col:
    lgb_train[col] = encoder.fit_transform(lgb_train[col])
    lgb_test[col] = encoder.transform(lgb_test[col])
    lgb_full_train[col] = encoder.fit_transform(lgb_full_train[col])
    lgb_full_test[col] = encoder.transform(lgb_full_test[col])
    cat_col_num.append(data.columns.to_list().index(col))

gbm = lgb.LGBMRegressor(boosting_type='gbdt', verbose=0, seed=SEED)


params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'n_estimators': [40, 60],
    'num_leaves': [21, 31, 41],
}

grid_gbm = GridSearchCV(gbm,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=True)

In [16]:
%%time
grid_gbm.fit(lgb_train, y_train_valid);

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  5.5min finished


CPU times: user 5min 29s, sys: 1.28 s, total: 5min 31s
Wall time: 5min 34s


GridSearchCV(cv=KFold(n_splits=3, random_state=21, shuffle=True),
             error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=No...
                                     reg_alpha=0.0, reg_lambda=0.0, seed=21,
                                     silent=True, subsample=1.0,
                                     subsample_for_bin=200000, subsample_freq=0,
                                     verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': array([0.001     , 0.00562341,

In [17]:
grid_gbm.best_params_

{'learning_rate': 0.1778279410038923, 'n_estimators': 60, 'num_leaves': 41}

In [28]:
res.append(model_eval(grid_gbm.best_estimator_, lgb_train, y_train_valid, lgb_test, y_valid))

#### CatBoost
1. Все тоже самое что и для прошлых моделей

In [19]:
cbr = CatBoostRegressor(random_seed=SEED,
                        loss_function='RMSE',
                        silent=True,
                        cat_features=cat_col)

params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'iterations': [40, 60],
    'depth': [6, 8, 10],
}

grid_cbr = GridSearchCV(cbr,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=False)

In [20]:
%%time
grid_cbr.fit(X_train_valid, y_train_valid);

CPU times: user 23min 15s, sys: 2min 52s, total: 26min 8s
Wall time: 30min 7s


GridSearchCV(cv=KFold(n_splits=3, random_state=21, shuffle=True),
             error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7f59025a50d0>,
             iid='warn', n_jobs=None,
             param_grid={'depth': [6, 8, 10], 'iterations': [40, 60],
                         'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=False)

In [21]:
grid_cbr.best_params_

{'depth': 10, 'iterations': 60, 'learning_rate': 0.1778279410038923}

In [30]:
res.append(model_eval(grid_cbr.best_estimator_, X_train_valid, y_train_valid, X_valid, y_valid))

# 3. Анализ моделей

Сведем все данные в один датафрейм и посмотрим результаты на валидационной выборке

Для выбранной модели посмотрим результат на тесте

In [31]:
pd.DataFrame(data=res,
             index=['Ridge', 'LightGBM', 'CatBoost'],
             columns=['trainig_time', 'predic_time', 'score'])

,trainig_time,predic_time,score
Ridge,6.332461,0.132203,2165.170417
LightGBM,5.099348,0.399870,1833.551857
CatBoost,36.000696,0.156596,1832.514801


В данном случае наиболее подходящей моделью, с оптимальным качеством и быстродействием оказался: LightGBM

Проверим метрику на тесте

In [34]:
model = models[1]
model.fit(lgb_full_train, y_train)
mse(y_test, model.predict(lgb_full_test))**0.5

1820.936835085372

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей